In [ ]:
%matplotlib widget

In [ ]:
from glob import glob
import numpy as np
import pandas as pd
import flammkuchen as fl
from split_dataset import SplitDataset
from bouter import Experiment
from fimpy.pipeline.general import calc_f0, dff
# from motions.utilities import stim_vel_dir_dataframe, quantize_directions
from scipy.interpolate import interp1d 
from scipy.signal import convolve2d
import colorspacious
import matplotlib.pyplot as plt
import json
from pathlib import Path

In [ ]:
# master = Path(r"Z:\Hagar and Ot\E0040\v10\LS")
master = Path(r'\\portulab.synology.me\data\Hagar and Ot\E0040\v10\LS')
fish_list = list(master.glob("*_f*"))
path = fish_list[2]
print(path)

In [ ]:
exp = glob(str(path / "*behavior*"))[0]
traces = fl.load(path / "filtered_traces.h5", "/detr")

In [ ]:
coords = fl.load(path / "data_from_suite2p_cells.h5", "/coords")
suite2p_brain = fl.load(path / "data_from_suite2p_cells_brain.h5")
in_brain_idx = suite2p_brain['coords_idx']

In [ ]:
with open(next(Path(path).glob("*metadata.json")), "r") as f:
        metadata = json.load(f)
lsconfig = metadata["imaging"]["microscope_config"]['lightsheet']['scanning']
z_tot_span = lsconfig["z"]["piezo_max"] - lsconfig["z"]["piezo_min"]
n_planes = lsconfig["triggering"]["n_planes"]
z_res = z_tot_span / n_planes
res = [0.6, 0.6, z_res]

In [ ]:
sensory_regressors = fl.load(path / "sensory_regressors.h5", "/regressors")
titles = ['right', 'backward right', 'backward', 'backward left', 'left', 'forward left', 'forward', 'forward right', ]

In [ ]:
fig, axs = plt.subplots(4, 8, figsize=(15, 12), gridspec_kw={'width_ratios': [6, 2, 6, 2, 6, 2, 6, 2], 'height_ratios': [1, 3, 1, 3]})
fig.subplots_adjust(left=0.05, wspace=0.3, hspace=0.3)

In [ ]:
for direction in range(0, 8, 2):
    print(direction)
    #traces = traces[:1600]
    current_dir = np.asarray(sensory_regressors.iloc[:, direction])
    #reg_values = current_dir @ traces[:, :] 
    num_traces = np.shape(traces)[1]
        
    reg_corr = np.zeros((num_traces))
    for i in range(num_traces):
        reg_corr[i] = np.corrcoef(current_dir, traces[:, i])[0,1]
       
    coords_ib = coords[in_brain_idx]
    reg_corr_ib = reg_corr[in_brain_idx]
        
    mp_ind = np.argsort(np.abs(reg_corr_ib))
    
    axs[1,direction].scatter(coords_ib[mp_ind,2]*res[0], coords_ib[mp_ind,1]*res[1], c=reg_corr_ib[mp_ind], s=2, alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)
    axs[1,direction+1].scatter(coords_ib[mp_ind,0]*res[2], coords_ib[mp_ind,1]*res[1], c=reg_corr_ib[mp_ind], s=2, alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)
    axs[0,direction].scatter(coords_ib[mp_ind,2]*res[0], coords_ib[mp_ind,0]*res[2], c=reg_corr_ib[mp_ind], s=2, alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)

    axs[0,direction].spines['right'].set_visible(False)
    axs[0,direction].spines['top'].set_visible(False)

    axs[1,direction+1].spines['right'].set_visible(False)
    axs[1,direction+1].spines['top'].set_visible(False)

    axs[1,direction].spines['right'].set_visible(False)
    axs[1,direction].spines['top'].set_visible(False)

    axs[0,direction+1].axis('off')

    axs[0,direction].set_title(titles[direction])


In [ ]:
for direction in range(1, 9, 2):
    print(direction)
    #traces = traces[:1600]
    current_dir = np.asarray(sensory_regressors.iloc[:, direction])
    #reg_values = current_dir @ traces[:, :] 
    num_traces = np.shape(traces)[1]
        
    reg_corr = np.zeros((num_traces))
    for i in range(num_traces):
        reg_corr[i] = np.corrcoef(current_dir, traces[:, i])[0,1]
       
    coords_ib = coords[in_brain_idx]
    reg_corr_ib = reg_corr[in_brain_idx]
        
    mp_ind = np.argsort(np.abs(reg_corr_ib))
    
    axs[3,direction-1].scatter(coords_ib[mp_ind,2]*res[0], coords_ib[mp_ind,1]*res[1], c=reg_corr_ib[mp_ind], s=2, alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)
    axs[3,direction].scatter(coords_ib[mp_ind,0]*res[2], coords_ib[mp_ind,1]*res[1], c=reg_corr_ib[mp_ind], s=2, alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)
    axs[2,direction-1].scatter(coords_ib[mp_ind,2]*res[0], coords_ib[mp_ind,0]*res[2], c=reg_corr_ib[mp_ind], s=2, alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)

    axs[2,direction-1].spines['right'].set_visible(False)
    axs[2,direction-1].spines['top'].set_visible(False)

    axs[3,direction].spines['right'].set_visible(False)
    axs[3,direction].spines['top'].set_visible(False)

    axs[3,direction-1].spines['right'].set_visible(False)
    axs[3,direction-1].spines['top'].set_visible(False)

    axs[2,direction].axis('off')

    axs[2,direction-1].set_title(titles[direction])


In [ ]:
fig.subplots_adjust(left=0.05, wspace=0.3, hspace=0.3)

In [ ]:
file_name = "corr map all 8 directions.jpg"
fig.savefig(path / file_name, dpi=300)
file_name = "corr map all 8 directions.pdf"
fig.savefig(path / file_name, dpi=300)